# Final Assigment for DTSA 5304 Fundamentals of Data Visualization

# Exploratory Visualization of Olympic Data from Year 2010 to 2016

## Data

Data for this analysis is taken from Kaggel's publically avaliable dataset "120 years of Olympic history: athletes and results".  Dataset has basic bio data on athletes and medal results from Athens 1896 to Rio 2016.  To restrict number of data point, only from year 2010 to 2016 data is considered for this analyis. 

#### Link of dataset : https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?select=athlete_events.csv

### Content of dataset 

The file athlete_events.csv contains 271116 rows and 15 columns. Each row corresponds to an individual athlete competing in an individual Olympic event (athlete-events). The columns are:

    1. ID - Unique number for each athlete
    1. Name - Athlete's name
    1. Sex - M or F
    1. Age - Integer
    1. Height - In centimeters
    1. Weight - In kilograms
    1. Team - Team name
    1. NOC - National Olympic Committee 3-letter code
    1. Games - Year and season
    1. Year - Integer
    1. Season - Summer or Winter
    1. City - Host city
    1. Sport - Sport
    1. Event - Event
    1. Medal - Gold, Silver, Bronze, or NA

For this Analysis, only below fields are considered :-
    1. Year - Integer
    1. Season - Summer or Winter
    1. Team - Team name
    1. NOC - National Olympic Committee 3-letter code
    1. Sex - M or F
    1. Sport - Sport
    1. Medal - Gold, Silver, Bronze, or NA

Also, data is filtered for :- 
1. Year 2010 to 2016 
1. Only rows having medal won  (where Medal != NaN). 
1. Data for Countries ('USA', 'RUS', 'GER','GBR','CHN','CAN','AUS','FRA','ITA','JPN', 'IND')

**Total 2960 data points are filtered for Analysis**

## Goal or Purpose 

Goal or Purpose of this analysis is to explore and visualize Olympic data available on Public domain to find various patterns in winning records of top few countries.

## Tasks 
  1. To see overview of winning records of top few countries.
  2. To drill down or zoom on various other parameters ( winning records per year,  Male Vs Female records, Winning records by Sport type, Winning record by Sport type). 
  



 

In [1]:
# Importing Modules. 

import altair as alt
import numpy as np
import pandas as pd


print('\n'.join(f'{m.__name__}={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))


# Removing Altair limitation of 5000 rows. Although, for this analysis, ensured data points are less than 5000.

alt.data_transformers.disable_max_rows()


altair=5.2.0
numpy=1.22.3
pandas=1.4.2


DataTransformerRegistry.enable('default')

In [2]:
# Dataset is downloaded from Kaggle site and kept at data directory.
# Link of Dataset : https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?select=athlete_events.csv

#\data\athlete_events.csv

df_all_years = pd.read_csv('data/athlete_events.csv')

In [3]:
# Data Wrangling :  Slicing and Filtering the dataset. 

# Extracting Only Columns ('Year','Season', 'Team', 'NOC', 'Sex','Sport','Medal') 
# Filtering data of "Year 2010 to 2016" &  Medal != NaN & Countries = ('USA', 'RUS', 'GER','GBR','CHN','CAN','AUS','FRA','ITA','JPN', 'IND')
 


df_olympic = df_all_years[['Year','Season', 'Team', 'NOC', 'Sex','Sport','Medal']]
year = [2010,2012,2014,2016]
noc = ['USA', 'RUS', 'GER','GBR','CHN','CAN','AUS','FRA','ITA','JPN', 'IND' ]
df_olympic= df_olympic.dropna()
df_olympic = df_olympic.query(f'Year=={year}')
df_olympic =df_olympic.query(f'NOC=={noc}')
#df_olympic =pd.to_datetime(df_olympic.Year, format='%Y')

# Changing Year column to Date type and taking only Year 
df_olympic['Year'] = pd.to_datetime(df_olympic['Year'], format='%Y')
df_olympic['Year'] = df_olympic['Year'].dt.strftime('%Y')

# Check the dataset
print(df_olympic.info()) 
print(df_olympic.head())
print(df_olympic.tail())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2960 entries, 158 to 271019
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    2960 non-null   object
 1   Season  2960 non-null   object
 2   Team    2960 non-null   object
 3   NOC     2960 non-null   object
 4   Sex     2960 non-null   object
 5   Sport   2960 non-null   object
 6   Medal   2960 non-null   object
dtypes: object(7)
memory usage: 185.0+ KB
None
     Year  Season           Team  NOC Sex           Sport   Medal
158  2016  Summer          Italy  ITA   M          Rowing  Bronze
174  2012  Summer         France  FRA   M        Handball    Gold
175  2016  Summer         France  FRA   M        Handball  Silver
278  2014  Winter  United States  USA   M  Figure Skating  Bronze
684  2012  Summer         Canada  CAN   F          Diving  Bronze
        Year  Season      Team  NOC Sex      Sport   Medal
270607  2010  Winter  Russia-1  RUS   M  Bobsleigh  Bronze
27060





# Final Visualization

## Summary of the key elements of your design and accompanying justification.

### Overall Design Principal Used / Final evaluation approach

1. Overview First : First overall wins of countries is visulaized then various other related stats are visualized. 
1. Zoom & Filter - Added Radio Selection for Country selections for Filtering and zooming for specific Country. 
1. Detail on Demand - Used Tooltip to Provide extra details when user hover mouse over Graph.

### Graph Details

These graphs are visulaize :- 
1. ByCountry - Medal Count By Country
1. ByYear - Medal Count By Year
1. BySex - Medal Count By Sex 
1. ByMedal - Medal Count By Type
1. BySport - Medal Count By Sport Type

### Mark and Encoding details

1. Mark type of bar is used for ByCountry, BySex , ByMedal, BySport graphs as bar type more suitable  for Ordered data to effectively visualize which country won more or less medal.
1.  Mark type point is used for ByYear graph for having 2D poistion scatter plot for effectively visualize coutry, number of medals won and year data. 
1.  Hue color pattern used for effectively distinguish categorical data 
1. Tooltip added to provide extra information on demand to make Visualization interactive.
1. Aggregation used to visalize the number of medals wons.

### Color Scheme
> **Hue** is used for Categorical Encoding to make sure each catogory is distinguisable 


### Selection Used
1. Radio type button used to select specific country.   Radio button is in genral media/device/platform friendly. Extra zooming and panning is not used to avoid desert frog phenomena.
  

In [4]:
# Create Selection 
country_list = df_olympic['NOC'].tolist() # Get list of Country from Pandas Dataframe  
country_list = list(set(country_list))  # Get Unique list of Country  
options = country_list
labels = [option + ' ' for option in options]

# Radio button for Country Selections.
input_dropdown = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ['All'],
    name='Country: '
)
selection = alt.selection_point(
    fields=['NOC'],
    bind=input_dropdown,
)

# Bar Graph of Total Medals won by each country 
ByCountry = alt.Chart(df_olympic, title="Medal Count By Country").mark_bar().encode(
    x = alt.X('NOC:N', sort = '-y', title = "Country"),
    y = alt.Y('count()', title = "Count of Medals" ),
    color=alt.Color('NOC',title="Legend Country List"),
     tooltip = [ 'Team' ,'count()'] 
).add_params(
    selection
).transform_filter(
    selection
)

 
#  Scatter Plot of total medals won each country per Year.
ByYear = alt.Chart(df_olympic, title="Medal Count By Year").mark_point().encode(
    x=alt.X('Year:T'),
    color=alt.Color('NOC:N').scale(domain=options),
     y = alt.Y('count()', title = "Count of Medals" ),
     tooltip = [ 'Team', "Year:T" ,'count()'],
     size="count()" 
).add_params(
    selection
).transform_filter(
    selection
)


#  Bar graph of total medal won by Male or Female. 
BySex = alt.Chart(df_olympic, title="Medal Count By Sex").mark_bar().encode(
    x = alt.X('count()', title = "Count of Medals" ),
    y = alt.Y('Sex:N', title = "Sex"),
    color=alt.Color('NOC:N').scale(domain=options),
    tooltip = [ 'Team' , 'Sex' , 'count()'] 
).add_params(
    selection
).transform_filter(
    selection
) 

#  Bar graph of medal won by type (Gold/Silver/Bronze). 
ByMedal = alt.Chart(df_olympic, title="Medal Count By Type").mark_bar().encode(
    x = alt.X('count()', title = "Count of Medals" ),
    y = alt.Y('Medal:N', title = "Medal Type"),
    color=alt.Color('NOC:N').scale(domain=options),
    tooltip = [ 'Team' , 'Medal' , 'count()'] 
).add_params(
    selection
).transform_filter(
    selection
) 

#  Bar graph of medal won by types of Sport. 
BySport = alt.Chart(df_olympic, title="Medal Count By Sport Type").mark_bar().encode(
    x = alt.X('Sport:N', title = "Sport"),
    y = alt.Y('count()', title = "Count of Medals" ),
    color=alt.Color('NOC:N'),
     tooltip = [ 'Team' , 'Sport' , 'count()'] 
).add_params(
    selection
).transform_filter(
    selection
) 



((ByCountry | ByYear ) & ( BySex | ByMedal )  & BySport) 

 
   

alt.VConcatChart(...)

#  Synthesis of findings

## Evaluation of the vizualization -  Formative Qualitative Evaluation
1. Online Survey done for Formative Qualitative Evaluation. 
1. Survey Link : https://forms.office.com/r/be1546V8TY
1. Kaggle Repo Link :https://www.kaggle.com/code/rahulchandrads/olympic-history-2010-16-exploratory-analysis
1. Participant :  Approached work colleague, family members, also posted survey link on Github and Kaggle. 
1. Below Questions Asked to check effectiveness of insight from vizualizations and its marks and encoding used  :
    1. Did Visulization help to gain insight of Olympic winning data ?
    1. Did selection of radio button effective to filter and explore individual country details?
    1. Is Color Scheme effective ?
    1. Did Tooltip help in intracting with visulization ?
    1. Were graph type (mark type) of bar chart and scatter chart effective to give insight of data ?
    1. Did overall Visulization meet your expectation ?


 ## Approach worked well 
  As per survey result, overall visualization is effective to give insight of olympic wining data. Overall principle of of "Overview First", "Zoom & Filter" and "Detail on Demand" woked well.  Bar chat and scatter plot are also able to repesent catorgoical data. 


  ## What elements can be  refined in future iterations. 
 1. Color - After Selecting any specific country. Same color is getting rendered for all graph. In next iteration, categorical Encoding can be applied for it. 
 1. Selection - More interval selection can be used for multi selection so that multiple countries can be selected to give more insight.